In [4]:
import pinecone
import tqdm.auto

In [5]:
pinecone.init(api_key="API_KEY_FROM_PINECONE", environment="gcp-starter")

In [6]:
#CREATE INDEX

In [9]:
pinecone.create_index("test", dimension=3, metric='dotproduct', pod_type='starter') 
#metrics to algorytm jaki ma byc uzywany do wyliczania odleglosci miedzy wektorami przy porownaniu czy euklidesowy dystans czy dot produkt itd
#pod_type wybor przestrzeni i zasobow S1/ P1/ P2

In [10]:
#retrive db list indexes

In [11]:
pinecone.list_indexes()

['test']

In [12]:
#connect to index

In [13]:
idx = pinecone.Index('test') 

In [14]:
#describe index

In [15]:
pinecone.describe_index('test')

IndexDescription(name='test', metric='dotproduct', replicas=1, dimension=3.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [19]:
print(pinecone.describe_index('test').status)
print(pinecone.describe_index('test').status["state"])

{'ready': True, 'state': 'Ready'}
Ready


In [20]:
#delete index

In [21]:
pinecone.delete_index('test')

In [23]:
###COLECTION - NOT ABLE TO QUERYING
# USEFUL FOR BACKUPS | MIGRATION | INDEX CONFIGURATION

In [24]:
pinecone.create_index('test', dimension=3, metric='dotproduct')

In [25]:
idx = pinecone.Index('test')
idx.upsert([('A', [1,2,3]),('B', [4,5,6])])

{'upserted_count': 2}

In [27]:
pinecone.create_collection(name='my-collection',source='test') #name of the coletion, source of collection

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '2a0cfc31b07b314830c00cfed256dbea;o=1', 'Date': 'Mon, 11 Dec 2023 21:37:16 GMT', 'Server': 'Google Frontend', 'Content-Length': '89', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: The requested feature 'Collections' is not supported by the current index type 'Starter'.


In [28]:
#list collection
pinecone.list_collections()

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '8afb97e50ca439c7e45596ecb518b1bf;o=1', 'Date': 'Mon, 11 Dec 2023 21:39:50 GMT', 'Server': 'Google Frontend', 'Content-Length': '89', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: The requested feature 'Collections' is not supported by the current index type 'Starter'.


In [ ]:
#Describe a collection
res = pinecone.delete_collection('my-collection') #output is an object to extract that why we set into res var
#we can then go through metadata of colleciton
res.dimension
res.vector_count
res.size

In [ ]:
#Deleting collection
pinecone.delete_collection('my-collection')

In [ ]:
#creating index from collection
pinecone.create_index(source_collection='my-collection')

In [30]:
###NAMESPACE

In [31]:
pinecone.list_indexes()

['test']

In [32]:
dimension = 3

In [33]:
emails_with_subject = 20
emails_with_body = 45
emails_with_other = 45

In [36]:
import numpy as np
vector_sub = np.random.rand(emails_with_subject, dimension).tolist() #(row, column) #pinecone require python list to work

In [37]:
vector_body = np.random.rand(emails_with_body, dimension).tolist()
vector_other = np.random.rand(emails_with_other, dimension).tolist()

In [47]:
#create ids for these vectors (it has to be str value for indexes that's why mapping is necessary)
ids_subjs = map(str, np.arange(emails_with_subject).tolist())
ids_body = map(str, np.arange(emails_with_body).tolist())
ids_other = map(str, np.arange(emails_with_other).tolist())

In [49]:
#zip id and vector
vector_subj = list(zip(ids_subjs, vector_sub)) #zip joins to its into tuple, then I wrap it into list of tuples

In [57]:
vectors_body = list(zip(ids_body, vector_body))
bectors_other = list(zip(ids_other, vector_other))

In [52]:
#upsert to index
idx.upsert(vector_subj, namespace="subject")

{'upserted_count': 20}

In [70]:
vectors_body = list(zip(map(str,np.arange(emails_with_body).tolist()),vector_body))

In [72]:
vectors_other= list(zip(map(str,np.arange(emails_with_other).tolist()),vector_other))

In [74]:
idx.upsert(vectors_body, namespace="body")
idx.upsert(vectors_other, namespace="other")

{'upserted_count': 45}

In [75]:
pinecone.delete_index('test')